In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
ENCODING_DICT = {
            'a': 0,
            'f': 1,
            'e': 2,
            'c': 3,
            'b': 4,
            'h': 5,
            'v': 6,
            'z': 7,
            '2': 8,
            'x': 9,
            'g': 10,
            'm': 11,
            'r': 12,
            'u': 13,
            'p': 14,
            's': 15,
            'd': 16,
            'n': 17,
            '6': 18,
            'k': 19,
            't': 20
            }

DECODING_DICT = {
            0: 'a',
            1: 'f',
            2: 'e',
            3: 'c',
            4: 'b',
            5: 'h',
            6: 'v',
            7: 'z',
            8: '2',
            9: 'x',
            10: 'g',
            11: 'm',
            12: 'r',
            13: 'u',
            14: 'p',
            15: 's',
            16: 'd',
            17: 'n',
            18: '6',
            19: 'k',
            20: 't'
            }

In [22]:
a = tf.constant([[1, 2, 3], [7, 8, 3]])
tf.reshape(a, [-1])

<tf.Tensor: shape=(6,), dtype=int32, numpy=array([1, 2, 3, 7, 8, 3], dtype=int32)>

In [65]:
def imagepath_to_captcha(imagepath):
    return imagepath.split("_")[1].split(".")[0]

def to_onehot(captcha):
    label = np.zeros((len(captcha), len(ENCODING_DICT)))
    for index, char in enumerate(captcha):
        label[index, ENCODING_DICT[char]] = 1
    return tf.convert_to_tensor(label, tf.float32)
    # return tf.reshape(tf.convert_to_tensor(label, tf.float32), [-1])

def captcha_from_onehot(tensor):
    label = []
    for index in tf.argmax(tensor, 1).numpy():
        label.append(DECODING_DICT[index])
    return "".join(label)

def load_dataset(filepath):
    filenames = os.listdir(filepath)
    x, y = [], []
    for file in filenames:
        imagepath = filepath + "/" + file
        image = tf.io.read_file(imagepath)
        image = tf.image.decode_image(image)
        pixels = tf.image.convert_image_dtype(image, tf.float32)
        # To show images: 
        # plt.imshow(tf.reshape(pixels, image.shape))
        # plt.title('Image as Pixels')
        # plt.show()
        x.append(pixels)
        captcha = imagepath_to_captcha(file).lower()
        y.append(to_onehot(captcha))
    return (tf.convert_to_tensor(x, tf.float32), tf.convert_to_tensor(y, tf.float32))


# Read input data

In [66]:
train_x, train_y = load_dataset("dataset/train")
test_x, test_y = load_dataset("dataset/test")

print("Training input shape:", train_x.shape)
print("Training output shape:", train_y.shape)
print("Testing input shape:", test_x.shape)
print("Testing output shape:", test_y.shape)

Training input shape: (8501, 50, 250, 3)
Training output shape: (8501, 6, 21)
Testing input shape: (1500, 50, 250, 3)
Testing output shape: (1500, 6, 21)


In [75]:
model = keras.models.Sequential()

model.add(keras.layers.Conv2D(128, (2, 2), activation='relu', input_shape=(50, 250, 3)))
# model.add(keras.layers.AveragePooling2D(2, 2))
# model.add(keras.layers.BatchNormalization())
# model.add(keras.layers.Conv2D(64, (2, 2), activation='sigmoid'))
# model.add(keras.layers.MaxPooling2D(2, 2))
# model.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
model.add(keras.layers.MaxPooling2D(2, 2))

model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(8064, activation='relu'))
# model.add(keras.layers.Dense(504, activation='relu'))
# model.add(keras.layers.Dense(504, activation=keras.layers.LeakyReLU(alpha=0.1)))
model.add(keras.layers.Dense(504, activation='relu'))
model.add(keras.layers.Dense(252, activation='relu'))
model.add(keras.layers.Dense(252, activation='relu'))
model.add(keras.layers.Dense(126, activation='softmax'))
model.add(keras.layers.Reshape((6, 21)))


model.summary()

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_43 (Conv2D)          (None, 49, 249, 128)      1664      
                                                                 
 max_pooling2d_26 (MaxPooli  (None, 24, 124, 128)      0         
 ng2D)                                                           
                                                                 
 flatten_14 (Flatten)        (None, 380928)            0         
                                                                 
 dense_61 (Dense)            (None, 504)               191988216 
                                                                 
 dense_62 (Dense)            (None, 252)               127260    
                                                                 
 dense_63 (Dense)            (None, 252)               63756     
                                                     

In [32]:
# ResNet50
model = keras.applications.ResNet50(
    include_top = False,
    weights = None,
input_tensor = keras.layers.Input(train_x))

TypeError: only integer scalar arrays can be converted to a scalar index

In [76]:
model.compile(
    optimizer = "adam",
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
)

# graphwiz doesn't install on gLinux ffs.
# tf.keras.utils.plot_model(model, show_shapes=True)


In [77]:
history = model.fit(train_x, train_y,
                    epochs = 5)

print(history.history)

Epoch 1/10


266/266 [==============================] - 101s 377ms/step - loss: 3.4777 - accuracy: 0.0585
Epoch 2/10
266/266 [==============================] - 97s 366ms/step - loss: 2.5350 - accuracy: 0.2147
Epoch 3/10
266/266 [==============================] - 97s 367ms/step - loss: 1.5280 - accuracy: 0.5050
Epoch 4/10
266/266 [==============================] - 102s 384ms/step - loss: 0.7949 - accuracy: 0.7379
Epoch 5/10
266/266 [==============================] - 103s 386ms/step - loss: 0.3998 - accuracy: 0.8679
Epoch 6/10
266/266 [==============================] - 102s 382ms/step - loss: 0.2418 - accuracy: 0.9213
Epoch 7/10
266/266 [==============================] - 103s 386ms/step - loss: 0.1563 - accuracy: 0.9505
Epoch 8/10
266/266 [==============================] - 103s 388ms/step - loss: 0.1266 - accuracy: 0.9594
Epoch 9/10
266/266 [==============================] - 102s 382ms/step - loss: 0.1109 - accuracy: 0.9651
Epoch 10/10
266/266 [==============================] - 103s 386ms/step - loss

In [78]:
model.evaluate(test_x, test_y)

47/47 [==============================] - 2s 39ms/step - loss: 2.6031 - accuracy: 0.5587


[2.603121519088745, 0.5586666464805603]